<a href="https://colab.research.google.com/github/Alfikriangelo/DeepLearningTasks/blob/main/3rdWeekTask/CNN_MLP_SVHN_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import library
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

In [2]:
# Load SVHN dataset
(ds_train, ds_test), ds_info = tfds.load('svhn_cropped', split=['train', 'test'], as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/svhn_cropped/incomplete.ZV9CIH_3.1.0/svhn_cropped-train.tfrecord*...:   0%…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/svhn_cropped/incomplete.ZV9CIH_3.1.0/svhn_cropped-test.tfrecord*...:   0%|…

Generating extra examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/svhn_cropped/incomplete.ZV9CIH_3.1.0/svhn_cropped-extra.tfrecord*...:   0%…

Dataset svhn_cropped downloaded and prepared to /root/tensorflow_datasets/svhn_cropped/3.1.0. Subsequent calls will reuse this data.


In [3]:
# Preprocessing
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train = ds_train.map(preprocess).batch(128).prefetch(1)
ds_test = ds_test.map(preprocess).batch(128).prefetch(1)

In [4]:
# CNN Model
cnn = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn.fit(ds_train, epochs=30, validation_data=ds_test)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 33s 43ms/step - accuracy: 0.4365 - loss: 1.6123 - val_accuracy: 0.8518 - val_loss: 0.4902
Epoch 2/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 26s 28ms/step - accuracy: 0.8376 - loss: 0.5318 - val_accuracy: 0.8799 - val_loss: 0.4032
Epoch 3/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - accuracy: 0.8708 - loss: 0.4268 - val_accuracy: 0.8984 - val_loss: 0.3495
Epoch 4/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 21s 25ms/step - accuracy: 0.8857 - loss: 0.3786 - val_accuracy: 0.9056 - val_loss: 0.3228
Epoch 5/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.8985 - loss: 0.3366 - val_accuracy: 0.9146 - val_loss: 0.3018
Epoch 6/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 16s 29ms/step - accuracy: 0.9044 - loss: 0.3178 - val_accuracy: 0.9204 - val_loss: 0.2847
Epoch 7/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 19s 26ms/step - accuracy: 0.9102 - loss: 0.2993 - val_accuracy: 0.9164 - val_loss: 0.2929
Epoch 8/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - accuracy: 0.9166 - loss: 0.2735 - 

In [5]:
# MLP Model
mlp = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(10, activation='softmax')
])

mlp.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=1
)

mlp.fit(
    ds_train,
    epochs=30,
    validation_data=ds_test,
    callbacks=[lr_schedule]
)

Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


573/573 ━━━━━━━━━━━━━━━━━━━━ 24s 33ms/step - accuracy: 0.2137 - loss: 2.4321 - val_accuracy: 0.1684 - val_loss: 3.3548 - learning_rate: 0.0010
Epoch 2/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 14s 25ms/step - accuracy: 0.4748 - loss: 1.5588 - val_accuracy: 0.3295 - val_loss: 2.1755 - learning_rate: 0.0010
Epoch 3/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.5028 - loss: 1.4928 - val_accuracy: 0.5598 - val_loss: 1.3457 - learning_rate: 0.0010
Epoch 4/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - accuracy: 0.5254 - loss: 1.4350 - val_accuracy: 0.5437 - val_loss: 1.4064 - learning_rate: 0.0010
Epoch 5/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - accuracy: 0.5450 - loss: 1.3912 - val_accuracy: 0.6079 - val_loss: 1.2376 - learning_rate: 0.0010
Epoch 6/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 14s 25ms/step - accuracy: 0.5605 - loss: 1.3571 - val_accuracy: 0.6150 - val_loss: 1.2164 - learning_rate: 0.0010
Epoch 7/30
573/573 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - accuracy: 0.5667 - loss: 1.3413

### Evaluasi Model Deep Learning

Model deep learning seperti **CNN (Convolutional Neural Network)** dan **MLP (Multilayer Perceptron)** dievaluasi dengan berbagai metrik performa. Berikut adalah rumus-rumus matematikanya:

---

### **1. Akurasi (Accuracy)**

Akurasi mengukur proporsi prediksi yang benar dari keseluruhan data:

$$
\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}
$$

**Keterangan:**
- \( TP \): True Positive (positif yang diprediksi benar)
- \( TN \): True Negative (negatif yang diprediksi benar)
- \( FP \): False Positive (negatif yang diprediksi positif)
- \( FN \): False Negative (positif yang diprediksi negatif)

---

### **2. Presisi (Precision)**

Presisi mengukur akurasi dari prediksi positif:

$$
\text{Precision} = \frac{TP}{TP + FP}
$$

---

### **3. Recall (Sensitivitas / True Positive Rate)**

Recall menunjukkan seberapa banyak data positif yang berhasil dideteksi oleh model:

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

---

### **4. F1 Score**

F1 Score adalah rata-rata harmonik dari presisi dan recall:

$$
F1 = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$

---

### **5. False Positive Rate (FPR)**

FPR digunakan dalam kurva ROC untuk menghitung tingkat kesalahan positif:

$$
\text{FPR} = \frac{FP}{FP + TN}
$$

---

### **6. Area Under Curve (AUC - ROC)**

AUC mengukur kemampuan model dalam membedakan antara kelas positif dan negatif:

$$
\text{AUC} = \int_0^1 \text{TPR}(x) \, dx
$$

Rumus ini merepresentasikan **luas di bawah kurva ROC**, di mana:

- $$ \text{TPR}(x) $$: True Positive Rate (Recall) sebagai fungsi dari False Positive Rate

---


In [6]:
# Evaluation Function
def evaluate_model(model, ds):
    y_true = []
    y_pred = []
    y_score = []

    for x, y in ds:
        preds = model.predict(x)
        y_true.extend(y.numpy())
        y_pred.extend(np.argmax(preds, axis=1))
        y_score.extend(preds)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_score = np.array(y_score)

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro')
    rec = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    auc = roc_auc_score(tf.keras.utils.to_categorical(y_true), y_score, multi_class='ovo')

    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")

    return acc, prec, rec, f1, auc

In [7]:
#CNN Evaluation
print("\nEvaluasi CNN:")
evaluate_model(cnn, ds_test)


Evaluasi CNN:
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/st

(0.9325061462814997,
 0.9245509625430252,
 0.9265865970830183,
 0.9253143509110835,
 np.float64(0.9947012623332927))

In [8]:
#MLP Evaluation
print("\nEvaluasi MLP:")
evaluate_model(mlp, ds_test)


Evaluasi MLP:
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step  
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/st

(0.6818531038721574,
 0.673332359640002,
 0.6503564762003358,
 0.658896313190422,
 np.float64(0.9309453708217477))

### Kesimpulan Evaluasi Model CNN dan MLP

Setelah dilakukan pelatihan dan pengujian pada dua arsitektur deep learning, yaitu **Convolutional Neural Network (CNN)** dan **Multilayer Perceptron (MLP)**, diperoleh hasil evaluasi sebagai berikut:

---

#### **1. Convolutional Neural Network (CNN)**

- **Accuracy**: 93.25%
- **Precision**: 92.46%
- **Recall**: 92.66%
- **F1 Score**: 92.53%
- **AUC (Area Under Curve)**: 99.47%


**Interpretasi:**
Model CNN menunjukkan performa yang sangat tinggi pada semua metrik evaluasi. Akurasi dan AUC yang mendekati sempurna menandakan bahwa model ini sangat andal dalam membedakan antara kelas. Tingginya nilai precision dan recall juga menunjukkan bahwa CNN mampu mengklasifikasikan data dengan tingkat kesalahan yang sangat rendah. Hal ini membuktikan bahwa CNN sangat efektif dalam mengenali pola kompleks, khususnya pada data citra yang memiliki struktur spasial.
---

#### **2. Multilayer Perceptron (MLP)**

- **Accuracy**: 68.19%
- **Precision**: 67.33%
- **Recall**: 65.04%
- **F1 Score**: 65.89%
- **AUC (Area Under Curve)**: 93.09%

**Interpretasi:**
Model MLP memperlihatkan performa yang jauh lebih rendah dibandingkan CNN. Nilai akurasi dan F1 score berada di bawah 70%, menunjukkan bahwa model ini kesulitan dalam menangkap pola dari data visual yang kompleks. Meskipun demikian, nilai AUC yang cukup tinggi (93.09%) masih mengindikasikan kemampuan model dalam membedakan antara kelas secara umum. Hasil ini memperkuat asumsi bahwa MLP lebih cocok diterapkan pada data non-visual atau data tabular yang tidak memiliki dimensi spasial.


---

### **Kesimpulan Umum**

- Model CNN terbukti unggul dalam seluruh metrik evaluasi dan menunjukkan performa yang sangat baik untuk tugas klasifikasi citra.
- Model MLP kurang efektif dalam menangani data visual, meskipun tetap menunjukkan potensi pada data dengan struktur yang lebih sederhana.
- Hanya model CNN yang berhasil melewati ambang batas akurasi 75%, menjadikannya sebagai pilihan utama dalam konteks klasifikasi data berbasis citra ini.

---

### Rekomendasi

Untuk tugas klasifikasi yang melibatkan data citra atau data dengan dimensi spasial, penggunaan arsitektur CNN sangat disarankan karena kemampuannya dalam mengekstraksi fitur spasial dengan sangat baik. Sebaliknya, MLP dapat menjadi alternatif yang lebih ringan secara komputasi untuk jenis data lain seperti data numerik atau tabular yang tidak memiliki kompleksitas spasial.

